In [1]:
from GSPTDataset import GSPTDataset
gspt = GSPTDataset(data_path="Global-Steel-Plant-Tracker-2023-03-2.xlsx",
                        missing_years_path="filled_missing_start_years.xlsx",
                        version_year=2023,
                        gspt2gspt_path="GSPT2GSPT.json")

In [2]:
from functions import get_cartesian, coordinates_change, f, transform_coordinates_to_lat_long, von_mises_mu
from numpy import linalg as LA

In [4]:
import src.utils
import pandas as pd
import numpy as np

from scipy.stats import vonmises_fisher
from matplotlib.colors import Normalize
from sklearn.metrics.pairwise import haversine_distances

In [3]:
df_gspt= gspt.get_operating_plants(start_year=2019,melt=True)
df_gspt=df_gspt.drop_duplicates(["Coordinates"])
df_gspt[['latitude', 'longitude']] = df_gspt['Coordinates'].str.split(',',expand=True)
df_gspt['latitude']=df_gspt['latitude'].astype(float)
df_gspt['longitude']=df_gspt['longitude'].astype(float)



c:\Users\Denisa.draghia\Desktop\Acier\GSPTDataset.py:538: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  steel_df[[c.replace("date", "year") for c in date_cols]] = steel_df.loc[:, date_cols].applymap(extract_year, na_action="ignore")


In [7]:
steel_factories_coordinates=df_gspt[['Country','Region','latitude','longitude']]
steel_factories_coordinates.to_csv("steel_factories_coordinates.csv")

In [8]:
mu=von_mises_mu(df_gspt)
mu

array([0.00885352, 0.62104601, 0.7837241 ])

In [11]:
transform_coordinates_to_lat_long([mu]) # 

array([51.602826, 89.183257])

In [12]:
rng = np.random.default_rng()

samples = vonmises_fisher(mu, 100).rvs(1, random_state=rng)
samples

array([[0.08022455, 0.66165571, 0.74550369]])

In [13]:
def f(coordinates_tirage, steel_coordinates,h):
    coordinates_lat_lon_tirage=transform_coordinates_to_lat_long(coordinates_tirage)
    def distance(a):
        return haversine_distances([a,coordinates_lat_lon_tirage])[0][1]
    distances=list(map(distance,steel_coordinates))
    return np.exp(-sum(distances)/h)
    
    


In [14]:
coordinates_change(df_gspt)

array([[  36.795924,    7.707633],
       [  36.753961,    6.24442 ],
       [  35.770912,   -0.263031],
       ...,
       [  37.272927,  -79.998828],
       [  47.569597, -122.367395],
       [  43.400533,  -87.9482  ]])

In [15]:
f(samples,coordinates_change(df_gspt),100) # précision de python : 1e-15

2.329870571230115e-06

In [16]:
multiple_samples_cartesian = vonmises_fisher(mu, 1).rvs(10**6, random_state=rng)
#transform_coordinates_to_lat_long(multiple_samples_cartesian[0][1])

d=multiple_samples_cartesian

In [17]:
vonmises_fisher.pdf(multiple_samples_cartesian,kappa=1, mu=np.array([0.00885352, 0.62104601, 0.7837241 ]))

array([0.17811912, 0.14197862, 0.11290058, ..., 0.08999047, 0.034197  ,
       0.13350492])

In [18]:
def transf(p):
    return transform_coordinates_to_lat_long(np.array([p]))

In [19]:
transform_coordinates_to_lat_long(np.array([[-0.00930249, -0.1878493 ,  0.98215381]]))

array([ 79.159274, -92.83503 ])

In [20]:
steel_coordinates=coordinates_change(df_gspt)

In [21]:
simulated_lat_lon=list(map(transf,multiple_samples_cartesian))


In [22]:
import scipy
from scipy import spatial

In [24]:
tuple_steel=[tuple(c) for c in steel_coordinates]

In [25]:
def geo(coordinate,h):
    
    def distance(tuple_steel_coordinate):
        return geodesic(coordinate,tuple_steel_coordinate).kilometers /6371 # rayon de la terre
    distances=list(map(distance,tuple_steel))
    return -sum(distances)/h

In [26]:
location=geolocator.reverse(simulated_lat_lon[3])
country_name = location.raw['address'].get('country', 'Unknown')
country_name

'Brasil'

In [29]:
usa=pd.read_hdf("C:\\Users\\Denisa.draghia\\Downloads\\climada_python-1.2.0\\climada_python-1.2.0\\data\\results\\5 minutes\\LitPop_pc_300_arcsec_USA_v1.hdf5")
usa=usa.drop(columns=['geometry','region_id','impf_'])
usa

,value,latitude,longitude
0,0.000000,51.375000,-178.958333
1,0.000000,51.875000,-178.208333
2,0.000000,51.875000,-178.125000
3,0.000000,51.875000,-178.041667
4,0.000000,51.875000,-177.958333
...,...,...,...
160780,0.000000,52.875000,173.041667
160781,4348.029150,52.875000,173.125000
160782,29688.926871,52.875000,173.208333
160783,0.000000,52.791667,172.791667


In [30]:
def geo(lat_lon,h,tuple_steel):
    lat, lon = lat_lon
    coordinate=np.array([lat,lon])
    def distance(tuple_steel_coordinate):
        return geodesic(coordinate,tuple_steel_coordinate).kilometers /6371 # rayon de la terre
    distances=list(map(distance,tuple_steel))
    return -sum(distances)/h

In [39]:
steel_coordinates=coordinates_change(df_gspt)
tuple_steel=[tuple(c) for c in steel_coordinates]
a=tuple_steel[5]
geo(list_usa[1],100,tuple_steel)

-9.68797524831873

In [ ]:
list_usa=[tuple(row) for row in usa.to_records(index=False)]
list_usa